In [ ]:
kf_outer = StratifiedKFold(n_splits=6)
lambdas = [0.01, 1, 100, 10000]
def cross_validation(X,y):
    outer_results = []
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []

    # First-level cross-validation
    for i, (train_index, test_index) in enumerate(kf_outer.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Inner (second-level) cross-validation for regularization parameter tuning
        kf_inner = StratifiedKFold(n_splits=5)
        inner_results = []
        
        for lambda_val in lambdas:
            lambda_scores = []
            
            for inner_train_index, inner_test_index in kf_inner.split(X_train, y_train):
                X_inner_train, X_inner_test = X_train[inner_train_index], X_train[inner_test_index]
                y_inner_train, y_inner_test = y_train[inner_train_index], y_train[inner_test_index]
                
                # Train and evaluate the model on the inner fold
                accuracy, _, _, _ = svm_classifier(X_inner_train, y_inner_train, X_inner_test, y_inner_test, lambda_val)
                lambda_scores.append(accuracy)
            
            # Store the average accuracy for this lambda value
            inner_results.append(np.mean(lambda_scores))
        
        # Select the best lambda value for this outer fold
        best_lambda = lambdas[np.argmax(inner_results)]
        
        # Evaluate the model on the outer fold using the selected lambda value
        accuracy, _, decision_statistics, _ = svm_classifier(X_train, y_train, X_test, y_test, best_lambda)

        # Calculate ROC curve and area under the curve
        fpr, tpr, thresholds = roc_curve(y_test, decision_statistics)
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        
        # Plot the ROC curve for the current fold
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

        outer_results.append(accuracy)
    
    # Compute the mean and standard deviation of the TPRs for the cross-validated ROC curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    # Plot the cross-validated ROC curve
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=0.8)

    # Plot the chance line
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=0.8)

    # Final touches to the plot
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Overt Dataset Cross Validation')
    plt.legend(loc="lower right")
    plt.show()
    
    # Compute the average accuracy across all outer folds
    average_accuracy = np.mean(outer_results)
    print(f"Average accuracy from two-level cross-validation: {average_accuracy}")
    #print best lambda value
    print(f"Best lambda value: {best_lambda}")

In [ ]:
#train on Imagined data and test on Overt data
X_train = np.concatenate((Img_data_1.T, Img_data_2.T))
y_train = np.array([0]*len(Img_data_1.T) + [1]*len(Img_data_2.T))
X_test = np.concatenate((Overt_data_1.T, Overt_data_2.T))
y_test = np.array([0]*len(Overt_data_1.T) + [1]*len(Overt_data_2.T))    
accuracy, channel_weights, decision_statistics, num_support_vectors = svm_classifier(X_train, y_train, X_test, y_test)
print("Accuracy:", accuracy)
print("Number of Support Vectors:", num_support_vectors)
print("Decision Statistics for all test samples:", decision_statistics)
plot_roc_curve(y_test, decision_statistics)
channel_weights_x = channel_weights[::2]
channel_weights_y = channel_weights[1::2]
show_channel_weights(channel_weights_x, channel_weights_y)